In [5]:
out_dir = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HEBSCHER_TMS/derivatives'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

# Option A - Generate Sphere Manually

In [3]:
#Enter coordinates of sphere
x = -30
y = -51
z = -9
radius=5

Generate the Sphere at The Coordinate

In [ ]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import calculate_voxelwise_mni_coords, calculate_pairwise_distance, generate_sphere_mask, points_within_sphere, save_mask, mask_within_brain
mni_coords_brain = calculate_voxelwise_mni_coords(brain_img)
distances = calculate_pairwise_distance(mni_arr, mni_coords_brain)
points_in_sphere = points_within_sphere(distances, radius)
sphere_mask = generate_sphere_mask(points_in_sphere, brain_img)
sphere_mask = mask_within_brain(sphere_mask, brain_img)
img = save_mask(sphere_mask, brain_img, mni_arr, out_dir)
img

# Option B - Generate Spheres from a CSV

In [6]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HERMILLER_TMS/metadata/master_list.csv'
sheet = None #'master_list_proper_subjects'

In [7]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,SUBJECT,stimsite_TLRC,_change_memory_score__post_stim_vs__pre_stim_,Study,Notes
0,CNDR001,52 48 43,0.200000,"Hermiller et al., 2019, Brain & Behav",percent change in cued associative verbal reca...
1,CNDR002,46 66 38,1.000000,NaN,five consecutive days of stimulation (HCN-targ...
2,CNDR004,50 66 35,0.083333,NaN,versus sham stimulation
3,CNDR005,47 65 28,0.363636,NaN,vs sham stimulation
4,CNDR006,42 69 35,0.100000,NaN,NaN
5,CNDR007,44 63 40,1.500000,NaN,NaN
6,CNDR009,47 66 26,0.250000,NaN,NaN
7,CNDR010,44 62 42,0.076923,NaN,NaN
8,CNDR011,45 70 31,0.000000,NaN,NaN
9,CNDR012,47 63 25,0.333333,NaN,NaN



# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [8]:
data_df.columns

Index(['SUBJECT', 'stimsite_TLRC',
       '_change_memory_score__post_stim_vs__pre_stim_', 'Study', 'Notes'],
      dtype='object')

In [9]:
drop_list = ['SUBJECT']

In [10]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)

Create x,y,z columns if not present

In [11]:
coord_col = 'stimsite_TLRC'

In [12]:
import numpy as np 
import pandas as pd
try:
    data_df[coord_col] = data_df[coord_col].apply(lambda x: np.array(x.split(), dtype=int))
except:
    print('column already prepared')
data_df[['x_tal', 'y_tal', 'z_tal']] = pd.DataFrame(data_df[coord_col].tolist(), index=data_df.index)
data_df

,SUBJECT,stimsite_TLRC,_change_memory_score__post_stim_vs__pre_stim_,Study,Notes,x_tal,y_tal,z_tal
0,CNDR001,"[52, 48, 43]",0.200000,"Hermiller et al., 2019, Brain & Behav",percent change in cued associative verbal reca...,52,48,43
1,CNDR002,"[46, 66, 38]",1.000000,NaN,five consecutive days of stimulation (HCN-targ...,46,66,38
2,CNDR004,"[50, 66, 35]",0.083333,NaN,versus sham stimulation,50,66,35
3,CNDR005,"[47, 65, 28]",0.363636,NaN,vs sham stimulation,47,65,28
4,CNDR006,"[42, 69, 35]",0.100000,NaN,NaN,42,69,35
5,CNDR007,"[44, 63, 40]",1.500000,NaN,NaN,44,63,40
6,CNDR009,"[47, 66, 26]",0.250000,NaN,NaN,47,66,26
7,CNDR010,"[44, 62, 42]",0.076923,NaN,NaN,44,62,42
8,CNDR011,"[45, 70, 31]",0.000000,NaN,NaN,45,70,31
9,CNDR012,"[47, 63, 25]",0.333333,NaN,NaN,47,63,25


Convert Coordinates to Different Space

In [13]:
from calvin_utils.nifti_utils.convert_coordinates import convert_coordinates_in_df
# Perform the conversion
df_converted = convert_coordinates(data_df, 'x_tal', 'y_tal', 'z_tal', 'talairach_to_mni')
print(df_converted)

ImportError: cannot import name 'convert_coordinates' from 'calvin_utils.nifti_utils.convert_coordinates' (/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/calvin_utils_project/calvin_utils_project/calvin_utils/nifti_utils/convert_coordinates.py)

Generate spheres given x, y, and z columns

In [8]:
radius = 5
xcol = 'mni_x'
ycol = 'mni_y'
zcol = 'mni_z'
subject_col = 'subject'

In [ ]:
from calvin_utils.nifti_utils import generate_spherical_roi
generate_spherical_roi.generate_spherical_rois_from_df(data_df, xcol, ycol, zcol, subject_col, radius, mask_path, out_dir)

/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:807: UserWarning: Casting data from int32 to float32
  return resample_img(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:297: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:807: UserWarning: Casting data from int32 to float32
  return resample_img(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:297: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(
/Users/cu135/.local/lib/python3.10/site-packages/nilearn/image/resampling.py:807: UserWarning: Casting data from int32 to float32
  return resample_